In [1]:
!pip install pytorch-nlp

     |████████████████████████████████| 92kB 6.0MB/s 


In [0]:
import numpy as np # to handle matrix and data operation
import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable

from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split

from torchnlp.word_to_vector import FastText

from torch.nn.utils.rnn import pad_sequence

In [0]:
model_save_location = "TCM_7.pt"

In [4]:
vectors = FastText()
device = torch.device("cuda:0")

BATCH_SIZE = 96

wiki.en.vec: 6.60GB [14:10, 7.76MB/s]                            
100%|██████████| 2519371/2519371 [04:52<00:00, 8609.85it/s]


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def load_data():
    '''Load dataset, data cleaned using Kaggle method.'''
    df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Big Armor/toxic-train-kaggle-clean.csv")
    df["word_splits"] = df["word_splits"].apply(eval)
    df = df[(df["word_splits"].apply(len) > 0) & (df["word_splits"].apply(len) <= 560)]

    X_train, X_test, y_train, y_test = train_test_split(df["word_splits"], df.drop("word_splits", axis=1), random_state=99, test_size=0.15)

    X_train = X_train.values
    y_train = y_train.values

    X_test = X_test.values
    y_test = y_test.values

    batched_X_train = []
    batched_y_train = []

    i=0
    while (i+1) * BATCH_SIZE < len(X_train):
        batched_X_train.append(X_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE])
        batched_y_train.append(y_train[i*BATCH_SIZE:(i+1)*BATCH_SIZE])
        i+=1
    batched_X_train.append(X_train[i*BATCH_SIZE:])
    batched_y_train.append(y_train[i*BATCH_SIZE:])

    batched_X_test = []
    batched_y_test = []

    del X_train
    del y_train

    i=0
    while (i+1) * BATCH_SIZE < len(X_test):
        batched_X_test.append(X_test[i*BATCH_SIZE:(i+1)*BATCH_SIZE])
        batched_y_test.append(y_test[i*BATCH_SIZE:(i+1)*BATCH_SIZE])
        i+=1
    batched_X_test.append(X_test[i*BATCH_SIZE:])
    batched_y_test.append(y_test[i*BATCH_SIZE:])

    del X_test
    del y_test

    return batched_X_train, batched_y_train, batched_X_test, batched_y_test

batched_X_train, batched_y_train, batched_X_test, batched_y_test = load_data()

In [0]:
class ToxicClassifierModel(nn.Module):
    def __init__(self,
                 LSTM_UNITS = 128,
                 dropout_rate = 0.2,
                 hidden1Activation = F.relu,
                 hidden2Activation = F.relu,
                 #hidden1Size = 512,
                 #hidden2Size = 512
                 ):
        super(ToxicClassifierModel, self).__init__()
        self.dropout_rate = dropout_rate
        self.BiGRU = nn.GRU(300, hidden_size = LSTM_UNITS, bidirectional=True, num_layers=1)
        self.BiRNN = nn.RNN(input_size = 2 * LSTM_UNITS, hidden_size = LSTM_UNITS, bidirectional=True)
        self.hidden1 = nn.Linear(4 * LSTM_UNITS, 4 * LSTM_UNITS)
        self.hidden1Activation = hidden1Activation
        self.hidden2 = nn.Linear(4 * LSTM_UNITS, 4 * LSTM_UNITS)
        self.hidden2Activation = hidden2Activation
        self.hidden3 = nn.Linear(4 * LSTM_UNITS, 6)
    
    def forward(self, X):
        X = X.permute(0, 2, 1)
        X = F.dropout2d(X, self.dropout_rate, training=self.training)
        X = X.permute(0, 2, 1)
        
        X = self.BiGRU(X)
        
        X = self.BiRNN(X[0])
        
        X = X[0]
        
        X = torch.cat((torch.max(X, 1).values, torch.mean(X, 1)), 1)
        
        X = X.add(self.hidden1Activation(self.hidden1(X)))
        
        X = X.add(self.hidden2Activation(self.hidden2(X)))
        
        X = torch.sigmoid(self.hidden3(X))
        
        return X

class ToxicClassifierFitter():
    def __init__(self,
                 optimizer,
                 error,
                 model,
                 vectors,
                 device,
                 EPOCHS = 2,
                 seed_acc = 0.5,
                 save_checkpoint = True,
                 model_save_location = model_save_location
                 ):
        self.optimizer = optimizer
        self.error = error
        self.model = model
        self.EPOCHS = EPOCHS
        self.acc = seed_acc
        self.vectors = vectors
        self.device = device
        self.model_save_location = model_save_location
        self.save_checkpoint = save_checkpoint
    
    def accuracy(self, batched_X_test, batched_y_test):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(zip(batched_X_test, batched_y_test)):
            var_X_batch = Variable(torch.nn.utils.rnn.pad_sequence([ self.vectors[X] for X in X_batch]).permute(1,0,2)).float().to(self.device)
            var_y_batch = Variable(torch.from_numpy(y_batch)).float().to(self.device)
            output = self.model(var_X_batch)

            # Total correct predictions
            predicted = output.data.round()
            correct += (predicted == var_y_batch).sum()
            del var_X_batch
            del var_y_batch
            del output
            del predicted
            torch.cuda.empty_cache()
            
        self.acc = float(correct*100) / float(6 * BATCH_SIZE * len(batched_X_test))
            
        return self.acc

    def F1Score(self, batched_X_test, batched_y_test):
        preds = []
        truePreds = []
        for batch_idx, (X_batch, y_batch) in enumerate(zip(batched_X_test, batched_y_test)):
            var_X_batch = Variable(torch.nn.utils.rnn.pad_sequence([ vectors[X] for X in X_batch]).permute(1,0,2)).float().to(device)
            var_y_batch = Variable(torch.from_numpy(y_batch)).float().to(device)
            output = self.model(var_X_batch)

            preds = preds + [ round(float(x)) for X in output.data for x in X ]
            truePreds = truePreds + [ round(float(x)) for X in var_y_batch for x in X ]
            del var_X_batch
            del var_y_batch
            del output
            torch.cuda.empty_cache()
            
        return f1_score(truePreds, preds)

    def predict(self, batched_X_test, batched_y_test):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(zip(batched_X_test, batched_y_test)):
            var_X_batch = Variable(torch.nn.utils.rnn.pad_sequence([ self.vectors[X] for X in X_batch]).permute(1,0,2)).float().to(self.device)
            var_y_batch = Variable(torch.from_numpy(y_batch)).float().to(self.device)
            output = self.model(var_X_batch)

            # Total correct predictions
            predicted = output.data.round()
            del var_X_batch
            del var_y_batch
            del output
            del predicted
            torch.cuda.empty_cache()
            
        return predicted
    
    def fit(self, batched_X_train, batched_y_train, batched_X_test = None, batched_y_test = None):
        for epoch in range(self.EPOCHS):
            correct = 0
            for batch_idx, (X_batch, y_batch) in enumerate(zip(batched_X_train, batched_y_train)):
                var_X_batch = Variable(torch.nn.utils.rnn.pad_sequence([ self.vectors[X] for X in X_batch]).permute(1,0,2)).float().to(self.device)
                var_y_batch = Variable(torch.from_numpy(y_batch)).float().to(self.device)
                self.optimizer.zero_grad()
                output = self.model(var_X_batch)
                loss = self.error(output, var_y_batch)
                loss.backward()
                self.optimizer.step()

                # Total correct predictions
                predicted = output.data.round()
                correct += (predicted == var_y_batch).sum()
                #print(correct)
                if batch_idx % 50 == 0:
                    print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                        epoch, batch_idx*len(X_batch), len(batched_X_train), 100.*batch_idx / len(batched_X_train), loss.data, float(correct*100) / float(6 * BATCH_SIZE*(batch_idx+1))))
                del var_X_batch
                del var_y_batch
                del loss
                del output
                del predicted
                torch.cuda.empty_cache()

            torch.cuda.empty_cache()
            if self.save_checkpoint:
                acc2 = self.accuracy(batched_X_test, batched_y_test)
                
                print("Validation accuracy Score:", acc2)
                
                if acc2 > self.acc:
                    print("Saving best model...")
                    torch.save(self.model.state_dict(), self.model_save_location)
                 
                    self.acc = acc2

    def fitF1(self, batched_X_train, batched_y_train, batched_X_test = None, batched_y_test = None):
        for epoch in range(self.EPOCHS):
            preds = []
            truePreds = []
            for batch_idx, (X_batch, y_batch) in enumerate(zip(batched_X_train, batched_y_train)):
                var_X_batch = Variable(torch.nn.utils.rnn.pad_sequence([ self.vectors[X] for X in X_batch]).permute(1,0,2)).float().to(self.device)
                var_y_batch = Variable(torch.from_numpy(y_batch)).float().to(self.device)
                self.optimizer.zero_grad()
                output = self.model(var_X_batch)
                loss = self.error(output, var_y_batch)
                loss.backward()
                self.optimizer.step()

                preds = preds + [ round(float(x)) for X in output.data for x in X ]
                truePreds = truePreds + [ round(float(x)) for X in var_y_batch for x in X ]

                if batch_idx % 50 == 0:
                    print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t F1:{:.3f}%'.format(
                        epoch, batch_idx*len(X_batch), len(batched_X_train), 100.*batch_idx / len(batched_X_train), loss.data, f1_score(truePreds, preds)))
                del var_X_batch
                del var_y_batch
                del loss
                del output
                torch.cuda.empty_cache()
            
            torch.cuda.empty_cache()
            if self.save_checkpoint:
                acc2 = self.F1Score(batched_X_test, batched_y_test)
                
                print("Validation F1 Score:", acc2)
                
                if acc2 > self.acc:
                    print("Saving best model...")
                    torch.save(self.model.state_dict(), self.model_save_location)
                 
                    self.acc = acc2
                

def createFitter(LSTM_UNITS = 128,
                 #hidden1Size = 512,
                 #hidden2Size = 512,
                 dropout_rate = 0.2,
                 hidden1Activation = F.relu,
                 hidden2Activation = F.relu,
                 learning_rate=0.001,
                 beta_1 = 0.9,
                 beta_2 = 0.999,
                 amsgrad=False,
                 weight_decay=0,
                 epochs = 1,
                 model_save_location=model_save_location,
                 vectors=vectors
                 ):
    
    # get device
    
    model = ToxicClassifierModel(LSTM_UNITS = LSTM_UNITS,
                                 dropout_rate = dropout_rate,
                                 hidden1Activation = hidden1Activation,
                                 hidden2Activation = hidden2Activation,
                                 #hidden1Size = hidden1Size,
                                 #hidden2Size = hidden2Size
                                )
    model.to(device)
    
    # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(beta_1,beta_2), amsgrad=amsgrad, weight_decay=weight_decay)

    optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate, betas=(beta_1,beta_2), weight_decay=weight_decay)
    
    error = nn.BCELoss()
    
    # return final fitter 

    return ToxicClassifierFitter(optimizer, error,
                                 model,
                                 vectors,
                                 device,
                                 EPOCHS = epochs,
                                 model_save_location = model_save_location,
                                 save_checkpoint = True)

In [0]:
activation_dict = {'relu':F.relu
                  ,'leaky_relu':F.leaky_relu
                  ,'softmax':F.softmax
                  ,'selu':F.selu
                  ,'tanh':F.tanh
                  ,'sigmoid':torch.sigmoid
                  ,'elu':F.elu
                  }
                      
TCMFitter = createFitter(
                 LSTM_UNITS = 128,
                 dropout_rate = 0.2,
                 hidden1Activation = activation_dict["relu"],
                 hidden2Activation = activation_dict["sigmoid"],
                 learning_rate = 1e-3, #0.002683035186257151,
                 amsgrad = False,
                 weight_decay = 0,
                 epochs = 20
                 )

In [20]:
TCMFitter.model.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/Big Armor/TCM_2.pt"))

<All keys matched successfully>

In [21]:
TCMFitter.fitF1(batched_X_train, batched_y_train, batched_X_test, batched_y_test)

Epoch : 0 [0/1395 (0%)]	Loss: 0.041375	 F1:0.696%
Epoch : 0 [4800/1395 (4%)]	Loss: 0.063367	 F1:0.760%
Epoch : 0 [9600/1395 (7%)]	Loss: 0.030748	 F1:0.768%
Epoch : 0 [14400/1395 (11%)]	Loss: 0.035718	 F1:0.760%
Epoch : 0 [19200/1395 (14%)]	Loss: 0.084591	 F1:0.760%
Epoch : 0 [24000/1395 (18%)]	Loss: 0.040829	 F1:0.759%
Epoch : 0 [28800/1395 (22%)]	Loss: 0.038186	 F1:0.758%
Epoch : 0 [33600/1395 (25%)]	Loss: 0.037616	 F1:0.759%
Epoch : 0 [38400/1395 (29%)]	Loss: 0.040687	 F1:0.760%
Epoch : 0 [43200/1395 (32%)]	Loss: 0.042657	 F1:0.761%
Epoch : 0 [48000/1395 (36%)]	Loss: 0.027223	 F1:0.762%
Epoch : 0 [52800/1395 (39%)]	Loss: 0.029626	 F1:0.762%
Epoch : 0 [57600/1395 (43%)]	Loss: 0.024181	 F1:0.763%
Epoch : 0 [62400/1395 (47%)]	Loss: 0.062037	 F1:0.763%
Epoch : 0 [67200/1395 (50%)]	Loss: 0.030517	 F1:0.761%
Epoch : 0 [72000/1395 (54%)]	Loss: 0.028989	 F1:0.761%
Epoch : 0 [76800/1395 (57%)]	Loss: 0.066981	 F1:0.761%
Epoch : 0 [81600/1395 (61%)]	Loss: 0.047291	 F1:0.760%
Epoch : 0 [86400/13

In [0]:
torch.save(TCMFitter.model.state_dict(), "TCM_4.pt")

In [0]:
torch.cuda.empty_cache()